# Wires från dgn till gdb
## fas och topp för sig
## littera som attribut

In [12]:
import arcpy, os, glob
import pandas as pd

local_dir = r"C:\SVK_2024\pythonkörningar"
dgn_dir = r"Q:\Projekt\Analys_2024\DGN_2024"
gdb  = os.path.join(local_dir, "Wires_241120_test.gdb")

LGs_info_path = r"Q:\Projekt\Data_2024\styrfiler\veg_kanttrad_test.txt"
cvd_LEDNINGSGATA_path = r"U:\Projekt\Data\Geodataunderlag_SVK\cvd_LEDNINGSGATA.txt"

arcpy.env.overwriteOutput = True

def wires_to_gdb(row):
    # Ange i definitionen att row är en pandas serie
    LG = row["LG"]
    line = row["line"]
    littera = row["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    
    print(f"Påbörjar export av fas- och topplinor för {LG}_{line}, {littera}")
    
    dgn = os.path.join(dgn_dir, f"{LG}_{line}.dgn")
    arcpy.env.workspace = dgn_dir # Varför ska den ha dgn dir som workspace?
    
    # faslinor
    fc_fas = f"{LG}_{line}_fas"
    # Create a value table that will hold the input feature classes for Merge
    vTab_fas = arcpy.ValueTable()
    layer_fas = "Layer_fas"
    arcpy.Delete_management(layer_fas)
    arcpy.MakeFeatureLayer_management(dgn + '/Polyline', layer_fas, "\"Level\" = 1")
    vTab_fas.addRow(layer_fas)
        
    # topplinor
    fc_topp = f"{LG}_{line}_topp"
    # Create a value table that will hold the input feature classes for Merge
    vTab_topp = arcpy.ValueTable()
    layer_topp = "Layer_topp"
    arcpy.Delete_management(layer_topp)
    arcpy.MakeFeatureLayer_management(dgn + '/Polyline', layer_topp, "\"Level\" = 2")
    vTab_topp.addRow(layer_topp)
    
    # Merge the CAD features into one feature class
    arcpy.env.workspace = gdb
    
    # faslinor
    arcpy.Merge_management(vTab_fas, fc_fas)
    # Add and populate littera field
    arcpy.management.AddField(fc_fas, "Littera", "TEXT")
    arcpy.management.CalculateField(fc_fas, "Littera", "".join(("'",littera,"'")))
    arcpy.management.AddField(fc_fas, "LG", "LONG")
    arcpy.management.CalculateField(fc_fas, "LG", int(LG_code))
    arcpy.management.AddField(fc_fas, "lintyp", "TEXT")
    arcpy.management.CalculateField(fc_fas, "lintyp", "'ledare'")
    # domains
    arcpy.management.AssignDomainToField(fc_fas, "LG", "cvd_LEDNINGSGATA")
    arcpy.management.AssignDomainToField(fc_fas, "Littera", "cvd_LITTERA_LEDNING")
    
    # ta bort fält som inte ska användas DETTA BEHÖVER GÖRAS PÅ BÅDE FAS OCH TOPP MEN NUVARANDE 
    # UPPREPNING ÄR KLUMPIG
    keep_fields = ["LG", "Littera", "lintyp", "Shape", "Shape_Length", "OBJECTID"]
    all_fields = arcpy.ListFields(fc_fas)
    fields_to_delete = [field.name for field in all_fields if field.name not in keep_fields]
    if fields_to_delete:
        arcpy.DeleteField_management(fc_fas, fields_to_delete)   

    # Räkna faslinor, varna om inga finns
    arcpy.management.DefineProjection(fc_fas, sr)
    result = arcpy.management.GetCount(fc_fas)
    count = int(result[0])
    if count == 0:
        print(f"VARNING: Inga faslinor hittades i level 1 i {LG}_{line}.dgn")
    
    # topplinor
    arcpy.Merge_management(vTab_topp, fc_topp)
    # Add and populate littera field
    arcpy.management.AddField(fc_topp, "Littera", "TEXT")
    arcpy.management.CalculateField(fc_topp, "Littera", "".join(("'",littera,"'")))
    arcpy.management.AddField(fc_topp, "LG", "LONG")
    arcpy.management.CalculateField(fc_topp, "LG", int(LG_code))
    arcpy.management.AddField(fc_topp, "lintyp", "TEXT")
    arcpy.management.CalculateField(fc_topp, "lintyp", "'topplina'")
    # domains
    arcpy.management.AssignDomainToField(fc_fas, "LG", "cvd_LEDNINGSGATA")
    arcpy.management.AssignDomainToField(fc_fas, "Littera", "cvd_LITTERA_LEDNING")
    
    # ta bort fält som inte ska användas
    keep_fields = ["LG", "Littera", "lintyp", "Shape", "Shape_Length", "OBJECTID"]
    all_fields = arcpy.ListFields(fc_topp)
    fields_to_delete = [field.name for field in all_fields if field.name not in keep_fields]
    if fields_to_delete:
        arcpy.DeleteField_management(fc_topp, fields_to_delete)
    
    arcpy.management.DefineProjection(fc_topp, sr)

    # Räkna topplinor, varna om inga finns
    result = arcpy.management.GetCount(fc_topp)
    count = int(result[0])
    if count == 0:
        print(f"VARNING: Inga topplinor hittades i level 2 i {LG}_{line}.dgn")
        
    # Städa bort tillfälliga lager från minnet
    arcpy.env.workspace = dgn_dir
    arcpy.Delete_management(layer_fas)
    arcpy.Delete_management(layer_topp)
        
# Sätt detta i en main-funktion för tydlighet?
sr = arcpy.SpatialReference("SWEREF99_TM", "RH2000")
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.LG[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(len(df_cvd_LEDNINGSGATA))}

LGs_info.apply(wires_to_gdb, axis=1)


Påbörjar export av fas- och topplinor för LG006_1, UL29
VARNING: Inga faslinor hittades i level 1 i LG006_1.dgn
Påbörjar export av fas- och topplinor för LG006_2, UL4 S1-2


0    None
1    None
dtype: object

In [10]:
arcpy.env.workspace = gdb
result = arcpy.management.GetCount('LG006_1_fas')
count = int(result[0])
print(count)

72
